In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

## Linked Interactions

### Linked Panning

#### Panning   
* when multiple plots have ranges that stay in sync

In [4]:
from bokeh.layouts import gridplot

In [5]:
x = list(range(11))
y0, y1, y2 = x, [10-i for i in x], [abs(i-5) for i in x]

In [6]:
plot_options = dict(
    width = 250,
    plot_height = 250,
    tools = 'pan, box_zoom, reset'
)

In [7]:
# create a new plot
s1 = figure(**plot_options)
s1.circle(x, y0, size=10, color="navy")

# create a new plot and share both ranges
s2 = figure(x_range = s1.x_range, y_range = s1.y_range, **plot_options)
s2.triangle(x, y1, size = 10, color = "firebrick")

# create a new plot and share only one range
s3 = figure (x_range = s1.x_range, **plot_options)
s3.square(x, y2, size = 10, color = 'olive')

GlyphRenderer(id='1090', ...)

In [8]:
fig = gridplot([[s1, s2, s3]])

show(fig)

In [9]:
x_1 = list(range(11))
y_1_ = [ item % 2 for item in x_1]

In [10]:
y_2_ = [i ** 2 for i in x_1]

In [11]:
y_2_

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [13]:
# Interestingly, trying to use a copy of any of these lists in the plot will fail with an error


# create a new plot
p1 = figure(**plot_options)
p1.circle(x_1, y_1_, size=10, color="navy")

# create a new plot and share both ranges
p2 = figure(x_range = p1.x_range, y_range = p1.y_range, **plot_options)
p2.triangle(x_1, y_2_, size = 10, color = "firebrick")

fig = gridplot([[p1, p2]])

show(fig)

## Linked Brushing

The plots must share a data source

In [14]:
from bokeh.models import ColumnDataSource

In [15]:
x = list(range(-20, 21))
y0, y1 = [abs(xx) for xx in x], [xx**2 for xx in x]

In [16]:
#Create a column data source for the plots to share
data = dict(x = x, y0 = y0, y1 = y1)
source = ColumnDataSource(data)

In [17]:
TOOLS = 'box_select,lasso_select,help, reset'

In [18]:
# create a new plot and add a renderer
left = figure(tools = TOOLS, width=300, height=300)
left.circle('x', 'y0', source = source)

GlyphRenderer(id='1641', ...)

In [19]:
# create another plot and add a renderer
right = figure(tools = TOOLS,  width=300, height=300)
right.circle('x', 'y1', source = source)

GlyphRenderer(id='1676', ...)

In [20]:
fig = gridplot([[left, right]])

show(fig)

## Widgets

In [21]:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider

In [27]:
slider = Slider(start = 0, end = 10, value = 1, step = .1, title = 'Example Slider')

show(widgetbox(slider))

In [30]:
from bokeh.models.widgets import Select

#slect = Select()

#show(widgetbox(select))

## Custom JS Callbacks

In [31]:
from bokeh.models import TapTool, CustomJS, ColumnDataSource

In [36]:
callback = CustomJS(code = 'alert("You Tapped a circle")')

tap = TapTool(callback = callback)

fig = figure(plot_width=600, plot_height=300, tools=[tap, 'reset', 'box_select'])

fig.circle(x=[1, 2, 3, 4, 5], y=[2, 5, 8, 2, 7], size=20, alpha = 0.7)

show(fig)

## Slider widget example

* The example below shows an action attached to a slider that updates a data source whenever the slider is moved.

In [41]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

In [42]:
x = [x*0.005 for x in range(0, 201)]

In [43]:
source = ColumnDataSource(data = dict(x = x, y = x))

In [45]:
plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source = source, line_width = 3, line_alpha = 0.6)



GlyphRenderer(id='2896', ...)

In [46]:
slider = Slider(start = 0.1, end = 6, value = 1, step = .1, title = 'Power')



In [52]:
update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")



In [53]:
slider.js_on_change('value', update_curve)

show(column(slider, plot))

## Data Selection Example

In [73]:
#Break this down

from random import random

# Generate 50 between 0.0 and 1.0 random floats in x and y
x = [random() for x in range(500)]
y = [random() for y in range(500)]

# Generate a string, navy, 50 times in a list
color = ["navy"] * len(x)

In [60]:
s1 = ColumnDataSource(data=dict(x=x, y=y, color=color))

In [74]:
p = figure(plot_width=400, plot_height=400, tools="lasso_select, reset", title="Select Here")

In [75]:
# Create a figure usin the ColumnDataSource created
p.circle('x', 'y', color='color', size=8, alpha=0.4, source=s1, 
         selection_color="firebrick", selection_alpha=0.4)

GlyphRenderer(id='3515', ...)

In [76]:
# s2 data has 2 lists that both have 2 elements in them; [0,1] and [0.5, 0.5]
# This will draw a horizontal line across the canvs
s2 = ColumnDataSource(data=dict(xm=[0,1],ym=[0.5, 0.5]))
p.line(x='xm', y='ym', color="orange", line_width=5, alpha=0.6, source=s2)


GlyphRenderer(id='3523', ...)

In [64]:

s1.callback = CustomJS(args=dict(s1=s1, s2=s2), code="""
    var inds = s1.selected.indices;
    if (inds.length == 0)
        return;

    var ym = 0
    for (var i = 0; i < inds.length; i++) {
        ym += s1.data.y[inds[i]]
    }
    
    ym /= inds.length
    s2.data.ym = [ym, ym]

    // necessary becasue we mutated source.data in-place
    s2.change.emit();  
""")

show(p)

In [78]:
from bokeh.plotting import figure
from bokeh import events
from bokeh.models import CustomJS, Div, Button
from bokeh.layouts import column, row

import numpy as np


In [79]:
x = np.random.random(size=2000) * 100
y = np.random.random(size=2000) * 100

In [80]:
p = figure(tools="box_select")
p.scatter(x, y, radius=1, fill_alpha=0.6, line_color=None)

GlyphRenderer(id='3765', ...)

In [81]:
div = Div(width=400)
button = Button(label="Button", width=300)
layout = column(button, row(p, div))

In [82]:
# Events with no attributes
button.js_on_event(events.ButtonClick,  CustomJS(args=dict(div=div), code="""
div.text = "Button!";
""")) 

p.js_on_event(events.SelectionGeometry, CustomJS(args=dict(div=div), code="""
div.text = "Selection! <p> <p>" + JSON.stringify(cb_obj.geometry, undefined, 2);
"""))

show(layout)